__[Automated Bulk Downloads of Landsat-8 Data Products in Python](http://geologyandpython.com/get-landsat-8.html)__<br>

__[espa-bulk-downloader](https://github.com/USGS-EROS/espa-bulk-downloader)__

__[pyModis](http://www.pymodis.org/)__

### pyModis

In [1]:
import pymodis

qualitymodis and convertmodis_gdal modules not enabled, maybe Python GDAL is missing
WxPython missing, no GUI enabled


In [ ]:
pymodis.__file__

In [ ]:
pymodis.

### earthdatascience
[Work with MODIS remote sensing data in Python](https://www.earthdatascience.org/courses/earth-analytics-python/multispectral-remote-sensing-modis/modis-remote-sensing-data-in-python/)

[Error instalando geopandas](https://stackoverflow.com/questions/54734667/error-installing-geopandas-a-gdal-api-version-must-be-specified-in-anaconda/54734864#54734864)

In [ ]:
from glob import glob
import os
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

In [ ]:
data = et.data.get_data('cold-springs-fire')
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

In [ ]:
# Create list of MODIS rasters for surface reflectance
modis_bands_pre_list = glob('data/cold-springs-fire/modis/reflectance/07_july_2016/crop/*_sur_refl_b*.tif')

# Sort the list of bands
modis_bands_pre_list.sort()
modis_bands_pre_list

In [ ]:
# Create raster stack
modis_bands_pre_out = 'data/cold-springs-fire/outputs/modis_bands_pre.tif'
es.stack(modis_bands_pre_list, modis_bands_pre_out)

In [ ]:
# Open raster stack
with rio.open(modis_bands_pre_out) as src_modis_bands:
    modis_bands_pre = src_modis_bands.read(masked=True)
    modis_bands_bounds = src_modis_bands.bounds
    modis_bands_meta = src_modis_bands.profile
    modis_bands_crs = src_modis_bands.crs

In [ ]:
# Plot MODIS RGB
ep.plot_rgb(modis_bands_pre, rgb=[0, 3, 2], title='Surgace Reflectance \n MODIS RGB Bands')
plt.show()

In [ ]:
# Identify minimum and maximum values of band 1 (red)
print(modis_bands_pre[1].min(), modis_bands_pre[1].max())

In [ ]:
# Open fire boundary
fire_boundary_path = 'data/cold-springs-fire/vector_layers/fire-boundary-geomac/co_cold_springs_20160711_2200_dd83.shp'

In [ ]:
fire_boundary = gpd.read_file(fire_boundary_path)
fire_bound_sin = fire_boundary.to_crs(modis_bands_crs)
output_path = os.path.join('data', 'cold-springs-fire', 'outputs', 'modis_pre_lsn3')
if not os.path.exists(output_path):
    os.mkdir(output_path)
modis_pre_cropped_images = es.crop_all(modis_bands_pre_list, output_path, fire_bound_sin,
                                       overwrite=True)
modis_pre_crop, modis_pre_meta = es.stack(modis_pre_cropped_images)
modis_bands_pre_data = ma.masked_where(modis_pre_crop == -28672, modis_pre_crop)

In [ ]:
# Create a list of titles
titles = ['Red band', 'Near infrared (NIR) band', 'Blue/Green band', 'Green band',
          'Near infrared (NIR) band', 'Mid-infrared band', 'Mid-infrared band']

# Plot all bands individually
ep.plot_bands(modis_bands_pre_data, cols=3, title=titles, figsize=(10, 6))
plt.show()

In [ ]:
# Plot MODIS RGB
ep.plot_rgb(modis_bands_pre_data, rgb=[0, 3, 2],
            title='Cropped surface reflectance \n MODIS RGB bands')
plt.show()

In [ ]:
# Create a colors and titles list to use in the histogram
colors = ['r', 'k', 'b', 'g', 'k', 'y', 'y']
titles = ["Red Band", "Near Infrared (NIR) Band", "Blue/Green Band",
          "Green Band", "Near Infrared (NIR) Band",
          "Mid-infrared Band", "Mid-infrared Band"]

# Plot histogram
ep.hist(modis_bands_pre_data, colors=colors, title=titles, cols=2)
plt.show()

In [ ]:
# Scale values of MODIS imagery stack
modis_bands_pre_scaled = modis_bands_pre_data * 0.0001

In [ ]:
# Identify minimum and maximum values of scaled band 1 (red)
print(modis_bands_pre_scaled[1].min(), modis_bands_pre_scaled[1].max())

In [ ]:
# Create a colors and titles list to use in the histogram
colors = ['r', 'k', 'b', 'g', 'k', 'y', 'y']
titles = ["Red Band", "Near Infrared (NIR) Band", "Blue/Green Band", "Green Band",
          "Near Infrared (NIR) Band", "Mid-infrared Band", "Mid-infrared Band"]

# Plot histogram
ep.hist(modis_bands_pre_scaled, colors=colors, title=titles, cols=2)
plt.show()
